In [64]:
import json 
import requests
import pandas as pd
import numpy as np
import re

In [65]:
srdf = pd.read_csv("service_requests_Train-test_set.csv")
srdf_grouped = srdf[:2000].groupby( ["Resolution_Code"] ).head(15) # take a sample
sr_dict = {}

def clean_text(text):
        comp = re.compile(r'Image:[\S]*')
        text = comp.sub('', text)
        comp = re.compile(r'CD\W\w\w\w\W.*\d\d\d\dCST')
        text = comp.sub('', text)
        return text
    
for index, row in srdf_grouped.iterrows():
    rescode = row['Resolution_Code']
    if row['Resolution_Code'] not in sr_dict:
        sr_dict[rescode] = {
            'rescode': rescode,
            'para': [],
            'topics': [],
            'wordcloud': '',
            'related_rescodes': [],
        } 
    sr_dict[rescode]['para'].append(clean_text(row['symptom']))
#print(sr_dict)
# create dictionary of words to all text 

In [66]:
srdf_grouped

,sr_id,Created_Date,Resolution_Code,symptom
0,1-200016966951,10/27/2016 12:51,Reset_UPS,UPS fails to come up Image:NONE-NONE-NONE
1,1-202188076151,12/12/2016 0:05,Troubleshoot_HEMIT_TANK,CD Mon 12/12 0600CST Experienced a severe arti...
2,1-186899060621,3/21/2016 20:55,No structural problem found,CD Tue 3/22 0700CST Tech is requesting error l...
3,360154703,10/18/2012 0:00,No structural problem found,ENABLED DOSE MANAGEMENT FOR CUSTOMER.
4,1-174918638991,7/31/2015 21:25,Configure_ConnectPro,Question regarding modality worklist change. P...
5,1-205402405831,2/24/2017 18:26,Troubleshoot_HEMIT_TANK,still getting artifacts when tilting gantry fo...
6,1-184683709341,2/10/2016 14:48,No structural problem found,Needing some Ip changes made.
7,1-182969359501,1/13/2016 23:59,Reset_the_system_time_clock,It has come to our attention that when this ma...
8,1-200863773491,11/15/2016 17:28,Adjust_gantry_tilt_speed,the gantree is not coming on Direct connect
9,1-182424492456,1/6/2016 23:16,Troubleshoot_GANTRY_TOUCH_SENSOR,Plate for front cover is broke


In [96]:
class PredixClient:
    def __init__(self):
        self.token = 'eyJhbGciOiJSUzI1NiIsImtpZCI6IlU5V0VXIiwidHlwIjoiSldUIn0.eyJqdGkiOiIwMjI5NzFhZTI1ZjY0ZjFmYmZlMGM1MzYxOWQxOTExNSIsInN1YiI6ImFwcF9jbGllbnRfaWQiLCJzY29wZSI6WyJjbGllbnRzLnJlYWQiLCJ1YWEucmVzb3VyY2UiLCJvcGVuaWQiLCJ1YWEubm9uZSIsInByZWRpeC1hc3NldC56b25lcy4yNmJkZTcxYy1kYWFmLTRjNzItOTc1NS1iYjIxZjU4YzJkNWIudXNlciIsImh0dHBzOi8vcHJlZGl4LWFuYWx5dGljcy1jYXRhbG9nLXJlbGVhc2UucnVuLmF3cy11c3cwMi1wci5pY2UucHJlZGl4LmlvIiwiYW5hbHl0aWNzLnpvbmVzLjdkNGM0ZTNmLWI1YTUtNDFkOC05MThkLTliMzNmOGI0ZGJiMi51c2VyIl0sImNsaWVudF9pZCI6ImFwcF9jbGllbnRfaWQiLCJjaWQiOiJhcHBfY2xpZW50X2lkIiwiYXpwIjoiYXBwX2NsaWVudF9pZCIsImdyYW50X3R5cGUiOiJjbGllbnRfY3JlZGVudGlhbHMiLCJyZXZfc2lnIjoiM2Y0NDAyNWIiLCJpYXQiOjE1MzcyNzUwMDQsImV4cCI6MTUzNzMxODIwNCwiaXNzIjoiaHR0cHM6Ly8xNGMwZDI3Yy1iY2RmLTQ0MzItYmUyYy1hMDk2ZDNiYjk0NzcucHJlZGl4LXVhYS5ydW4uYXdzLXVzdzAyLXByLmljZS5wcmVkaXguaW8vb2F1dGgvdG9rZW4iLCJ6aWQiOiIxNGMwZDI3Yy1iY2RmLTQ0MzItYmUyYy1hMDk2ZDNiYjk0NzciLCJhdWQiOlsiaHR0cHM6Ly9wcmVkaXgtYW5hbHl0aWNzLWNhdGFsb2ctcmVsZWFzZS5ydW4uYXdzLXVzdzAyLXByLmljZS5wcmVkaXgiLCJjbGllbnRzIiwidWFhIiwib3BlbmlkIiwiYW5hbHl0aWNzLnpvbmVzLjdkNGM0ZTNmLWI1YTUtNDFkOC05MThkLTliMzNmOGI0ZGJiMiIsInByZWRpeC1hc3NldC56b25lcy4yNmJkZTcxYy1kYWFmLTRjNzItOTc1NS1iYjIxZjU4YzJkNWIiLCJhcHBfY2xpZW50X2lkIl19.c1YC2YPq4lTQ5-a9wDeu2SNDwFFghqmi55WScXwSDb_bNwIk7oZrXhCZArslXDijoob3azyW-LZQBWxbZHpQs8SG-GxD2Skev-bzTAgj-TgAW_hT2QJYLriiMtx95KInk_l-h3Bqja1Nart0WGinxB_TFH4hEKcmb-jU3fbZvw3kuejensl9J-5N8qvR-uZm8bxYJ-MQTGS18_GfgICLefd0HY8k21cF96ULCAUr502bS5gSNwvca08Crp__FR87dK2K2lW9BxhOnHmCKN7GAzeqrjFoNli3-UHxS1hG03iQWtz8QI0Ltm0waW94qp-4FJW9vq2DymuK8vlFDaFAeg'
        
    def run_rake(self, text):
        analytics_id = 'f7327915-7f5c-417f-b798-cae17771715a'
        res = requests.post('https://predix-analytics-catalog-release.run.aws-usw02-pr.ice.predix.io/api/v1/catalog/analytics/{}/execution'.format(analytics_id),
                      headers={
                          'Content-Type': 'application/json',
                          'Authorization': 'Bearer {}'.format(self.token),
                          'Predix-Zone-Id': '7d4c4e3f-b5a5-41d8-918d-9b33f8b4dbb2'
                      },
                      json={
                          'input_test': text
                      }
                     )
        return ' '.join(json.loads(res.json()['result']))
    
    def get_wordcloud(self, text):
        analytics_id = '37a38328-d235-42d3-9de4-3b9ccfed8f02'
        res = requests.post('https://predix-analytics-catalog-release.run.aws-usw02-pr.ice.predix.io/api/v1/catalog/analytics/{}/execution'.format(analytics_id),
                            headers={
                                'Content-Type': 'application/json',
                                'Authorization': 'Bearer {}'.format(self.token),
                                'Predix-Zone-Id': '7d4c4e3f-b5a5-41d8-918d-9b33f8b4dbb2'
                            },
                            json={
                                "config": {"wordcloud": "True"},
                                "data":   {"text" : text}
                            }
                           )
        print(json.loads(res.json()['result']))
        return json.loads(res.json()['result'])['wordcloud_img']
    
    def get_topics(self, text):
        analytics_id = '686b0ab7-f124-4731-9861-b66227d2ee60'
        res = requests.post('https://predix-analytics-catalog-release.run.aws-usw02-pr.ice.predix.io/api/v1/catalog/analytics/{}/execution'.format(analytics_id),
                            headers={
                                'Content-Type': 'application/json',
                                'Authorization': 'Bearer {}'.format(self.token),
                                'Predix-Zone-Id': '7d4c4e3f-b5a5-41d8-918d-9b33f8b4dbb2'
                            },
                            json={
                                "wotd_data": {
                                    "description": text
                                },
                                "parameters": {
                                    "n_topics" : 1,
                                    "n_words" : 10,
                                    "col_names":["description"],
                                    "top_n": 1
                                }
                            }
                           )
        return json.loads(res.json()['result'])['Topic'][0]
    
    def run_similarity(self, sr, rescode):
        rescode_list = list(sr.keys())
        # FIXME: there's a limit to input size for calling the predix service, hence only using top 3 topics
        topics = [' '.join(sr[x]['topics'].split(' ')[:3]) for x in rescode_list]
        analytics_id = 'b56ac7e1-ce70-45a0-9b68-3c8375039a3c'
        res = requests.post('https://predix-analytics-catalog-release.run.aws-usw02-pr.ice.predix.io/api/v1/catalog/analytics/{}/execution'.format(analytics_id),
                            headers={
                                'Content-Type': 'application/json',
                                'Authorization': 'Bearer {}'.format(self.token),
                                'Predix-Zone-Id': '7d4c4e3f-b5a5-41d8-918d-9b33f8b4dbb2'
                            },
                            json={
                                "data": {
                                    "data_set": {
                                        "rescode": rescode_list,
                                        "topics": topics
                                    },
                                    "constants":{
                                        "ref_id_column":"rescode",
                                        "desc_column":"topics",
                                        "query_ref_id":rescode,
                                        "cutoff":"0.05", # FIXME: lower cutoff to get more results
                                        "max_search_radius":"3"
                                    }
                                }
                            }
                           )
        return res.json()['result']
    
pc = PredixClient()

In [20]:
# visualize words on word cloud 
for rescode in sr_dict:
    print(rescode)
    sr_dict[rescode]['wordcloud'] = pc.get_wordcloud(sr_dict[rescode]['para'])


Reset_UPS
{'wordcloud_img': '...', 'tf': {'row': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9], 'col_map': {'eaton': 14, 'weekend': 65, 'after': 3, 'network': 32, 'off': 39, 'this': 56, 'replacement': 47, 'to': 57, 'the': 54, 'unit': 60, 'none': 34, 'failed': 16, 'panel': 43, 'whole': 66, 'can': 10, 'battery': 6, 'imus': 24, '5637': 0, 'it': 28, 'message': 30, 'fuse': 19, 'is': 27, 'up': 61, 'fails': 17, 'outage': 41, 'enabled': 15, 'in': 25, 'back': 5, 'getting': 21, 'scanner': 49, 'breaker': 8, 'states': 51, 'no': 33, 'patient': 44, 'unable': 59, 'fe': 18, 'reset': 48, 'will': 67, 'power': 46, 'ups': 62, 'get': 20, 'come': 11, 'a1': 2, 'lights': 29, 'on': 40, 

{'wordcloud_img': '...', 'tf': {'row': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9], 'col_map': {'arcing': 8, 'part': 48, 'carpet': 15, 'stops': 65, 'series': 57, 'exam': 24, 'intermittent': 38, 'severe': 59, 'to': 70, 'the': 67, 'slices': 61, '42': 4, '41': 3, 'noted': 43, 'hardware': 30, 'need': 41, 'sweeps': 66, 'cannot': 14, 'hemit': 35, 'recommit': 50, 'data': 20, 'scans': 55, 'committed': 17, 'asap': 11, '22680': 2, 'head': 32, 'us': 72, '203961962661': 1, 'appearing': 7, 'tilting': 69, 'experienced': 25, 'but': 13, 'downtime': 21, 'getting': 29, 'tilted': 68, 'scanner': 54, 'unknown': 71, 'sr': 62, 'states': 63, 'scan': 52, 'scanned': 53, 'service': 58, 'quality': 49, 'will': 74, 'clear': 16, 'second': 56, 'openion': 47, 'image': 36, 'and': 6, 'dur

{'wordcloud_img': '...', 'tf': {'row': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9], 'col_map': {'they': 95, 'making': 56, 'after': 8, '19th': 1, 'dose': 27, 'noise': 62, 'changes': 19, 'error': 31, 'qa': 72, 'up': 100, 'system': 89, 'configuration': 21, 'to': 98, '360': 4, 'certificate': 18, 'and': 11, 'states': 87, 'tube': 99, 'some': 85, 'was': 103, 'hardware': 43, 'restart': 75, 'an': 10, '0582': 0, 'routine': 77, 'normal': 63, 'issues': 50, 'logs': 54, 'at': 13, 'ilinq': 46, 'dosewatch': 28, 'called': 17, 'made': 55, 'it': 51, 'console': 22, 'ip': 47, 'due': 29, 'is': 48, 'than': 93, 'call': 16, 'locked': 53, 'fastcals': 32, 'new': 60, 'amy': 9, 'the': 94, 'enabled': 30, 'h

{'wordcloud_img': '...', 'tf': {'row': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9], 'col_map': {'craig': 23, 'place': 74, 'after': 7, 'down': 33, 'shut': 93, 'dose': 32, 'information': 52, 'help': 48, 'service': 91, 'prior': 80, 'this': 101, 'modality': 59, 'split': 94, 'used': 108, 'to': 103, 'the': 100, 'updating': 106, 'planned': 75, 'wont': 112, 'need': 61, 'howard': 49, 'report': 84, 'set': 92, 'select': 88, 'restarted': 86, 'date': 26, 'asked': 11, '08': 3, '07': 2, 'caller': 16, 'sent': 90, '00': 0, 'change': 19, 'populating': 77, 'pps': 78, 'at': 12, '

{'wordcloud_img': '...', 'tf': {'row': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9], 'col_map': {'would': 94, 'they': 83, '23': 1, '15': 0, 'come': 24, 'also': 9, 'needs': 57, 'this': 84, 'minutes': 52, 'help': 40, 'clocks': 23, 'mountain': 54, 'savings': 67, 'system': 78, 'am': 10, 'to': 86, 'the': 81, '42': 2, 'friday': 35, 'remote': 63, 'missing': 53, 'it': 48, 'slow': 74, 'fix': 32, 'date': 27, 'can': 20, 'sharon': 72, 'hrs': 44, 'machine': 51, 'at': 15, 'its': 49, 'hospital': 42, 'fixed': 33, 'clock': 22, 'change': 21, 'is': 47, 'someone': 76, 'time': 85, 'up': 88, 'wall

...

{'wordcloud_img': '...', 'tf': {'row': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9], 'col_map': {'they': 113, 'que': 87, 'making': 66, 'crunching': 29, 'detector': 34, 'down': 37, 'shut': 99, 'them': 112, 'august': 13, 'anywhere': 10, 'box': 17, 'to': 115, 'prompt': 84, 'r64494': 88, 'listed': 64, 'cannot': 22, 'can': 21, 'abort': 5, 'studies': 107, 'machine': 65, 'reformat': 90, 'says': 94, 'freezing': 43, 'it': 58, 'reconstructed': 89, 'is': 57, 'from': 44, 'see': 97, 'in': 55, 'line': 62, 'works': 126, '17461':

{'wordcloud_img': '...', 'tf': {'row': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9], 'col_map': {'which': 87, 'attempting': 9, 'present': 58, 'functioning': 33, 'come': 16, 'spine': 71, 'ext': 28, 'error': 26, 'this': 77, 'exam': 27, 'longer': 42, 'split': 72, '516': 3, 'confirm': 18, 'to': 78, 'the': 76, 'upgrade': 83, 'aviable': 10, 'wont': 89, 'none': 49, 'missing': 45, 'info': 37, 'software': 70, 'during': 23, 'function': 32, 'restarted': 61, 'use': 84, 'working': 91, 'pps': 56, 'send': 66, 'fmi': 30, 'installed': 38, 'message': 44, 'more': 46, 'roi': 62, 'is': 39, 'call': 13, 'up': 82, 'description': 20, 'sec': 65, 'completed': 1

{'wordcloud_img': '...', 'tf': {'row': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9], 'col_map': {'gepacs': 37, 'they': 94, 'acquired': 10, '14': 2, 'destination': 25, 'speak': 84, '32768': 9, 'put': 70, 'error': 30, 'this': 95, 'lost': 49, 'exam': 31, 'setenv': 83, 'box': 14, 'direct': 26, 'gantry_type': 36, 'requested': 75, 'to': 96, 'the': 93, '13hw38': 1, 'nio64': 52, '1964233660': 5, 'ct': 20, 'anything': 12, 'need': 51, 'cannot': 16, 'pinq': 63, 'distorted': 27, 'rte': 77, 'optima': 57, 'doing': 29, 'use': 102, 'sent': 82, 'who': 107, 'database': 23, 'scans': 79, 'consoletype': 19, 'send': 81, 'ca

{'wordcloud_img': '...', 'tf': {'row': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9], 'col_map': {'normal': 54, 'they': 77, 'scanning': 65, 'breathing': 15, 'instructions': 40, 'problem': 61, 'discussed': 27, 'cst': 24, 'control': 23, 'direct': 26, 'to': 79, 'the': 76, 'and': 9, 'values': 82, 'loud': 47, 'cannot': 18, 'taking': 75, 'limited': 45, 'insite': 39, 'auto': 13, 'even': 31, 'can': 17, 'sent': 66, 'working': 89, '0700': 1, '09': 2, 'scan': 64, 'at': 11, 'noise': 51, 'engineer': 30, 'wants': 86, 'intercom': 41, 'panel': 58, 'settings': 68, 'occurred': 56, 'is': 43, 'speaker': 70, 'password': 59, 'involved': 42, 'log': 46, 'noisy': 52, 'hear': 35, 'in': 37, 'but': 16, 'reports': 62, 'sr': 7

KeyboardInterrupt: 

In [68]:
# get topics from topic detector 
counter = 0
for rescode in sr_dict:
    print(rescode)
    sr_dict[rescode]['topics'] = pc.get_topics(sr_dict[rescode]['para'])
    counter += 1


Reset_UPS
{'analyticId': '686b0ab7-f124-4731-9861-b66227d2ee60', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"wotd_data": {"description": ["UPS fails to come up ", "Site power outage yesterday. UPS in bypass mode and battery breaker will not stay on.", "No power to the whole system. A1 panel has lights.", "Customer states back up unit UPS is not coming/Fuse blown message Fe to obtain PO #", "SYSTEM UPS WAS NOT ENABLED/OFF. I TURNED IT ON AND NOW THIS SCANNER CAN NOT NETWORK IMAGES TO IMUS.   5637-NONE-NONE", "Replacement of Eaton 9155 UPS", "Customer states the unit is not coming up  thinks it is the UPS", "Customer states unable to get scanner to come up/ups is off", "Power outage over the weekend and the UPS battery breaker will not stay on UPS in bypass mode.", "System not getting ON after power outage. Failed to reset Power ware 9155 UPS. no patient involved.", "System keeps losing power.", " After a restart it froze and will not come back up

{'analyticId': '686b0ab7-f124-4731-9861-b66227d2ee60', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"wotd_data": {"description": ["the gantree is not coming on  Direct connect", " System is not reconstructing all of the images on every exam. System also only wants to tilt a half of a degree at a time on head scans until it gets to about 7 degrees and then it will tilt the rest of the way. ", "gantry wont tilt", " THE GANTRY ANGLE WILL NOT COMPLETELY UNDO ITSELF AFTER A CT WITH AN ANGLE ", "NO", "Tilt speed errors", "Gantry tilt issue"]}, "parameters": {"n_topics": 1, "n_words": 10, "col_names": ["description"], "top_n": 1}}', 'result': '{"Score": [100.0], "Topic ID": [1], "Topic": ["TILT GANTRY SYSTEM ANGLE UNDO SPEED ERRORS HEAD TIME WANTS"]}', 'createdTimestamp': '2018-09-18 17:05:59.618', 'updatedTimestamp': '2018-09-18 17:06:08.435'}
Troubleshoot_GANTRY_TOUCH_SENSOR
{'analyticId': '686b0ab7-f124-4731-9861-b66227d2ee60', 'status': 'COMPLETED', 

{'analyticId': '686b0ab7-f124-4731-9861-b66227d2ee60', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"wotd_data": {"description": ["CD Thur 9/22 0700CST FMI completed 9/20.  Exam split function no longer working since upgrade.  When attempting to exam split  get error message that PPS function is unable to send.   ", "flouro doesn\'t work. please refer to ge system log for more info for more reconfirm rx ", "Dynamic Transition not functioning.  Dynamic Transition is enabled but the option to choose a transition ROI is not present during Smart Prep Set up.  Please call 516-256-6200 ext 3994 ", "Need to confirm which software options are currently installed on this system.", "Exam Split option is aviable but will not allow for splitting.  Error about Code Meaning and Description ", "exam split is not working   will not load.   system was restarted ", "Exam Split is not working.", "wont exam split images to pacs. unknown pps  spine-NONE-NONE", "Exam s

{'analyticId': '686b0ab7-f124-4731-9861-b66227d2ee60', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"wotd_data": {"description": ["Clicking sound coming from gantry when it spins slow.", "Gantry noise when rotating for scout images.  Axial encoder noise ", "PM Followup", "Had a ring artifact", "Ticking sound as Gantry rotates", "KNOCKING NOISE FROM INSIDE THE GANTRY WHILE MACHINE IS RUNNING FOR SCAN ", "Gantry encoder requires replacement.", "CD Tues 9/8  at 13:24 error message of scanner hardware stopped scan.  case number 10704 scanner rebooted  again got same error message at 13:32 ", "encoder is noisy", "THE SCANNER FROZE DURING A SCAN AND WILL NOT ALLOW US TO CONTINUE THE SCAN.  THE PATIENT HAD TO BE TAKEN OFF AND MOVED TO ANOTHER ROOM ", "Direct connect CD Thursday 11/06 0700am. Customer State is getting a error messages.", "Noisy gantry encoder"]}, "parameters": {"n_topics": 1, "n_words": 10, "col_names": ["description"], "top_n": 1}}', 're

{'analyticId': '686b0ab7-f124-4731-9861-b66227d2ee60', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"wotd_data": {"description": ["Unable to do fast cals", "Hardware stops scan", "CD (E) 06/22/2015 08:00:00artifact on ct head  ", "Scan paused during scan 48258; resumed and finished scan; scan paused during scout series on 48259 ", "multiple times when scanner aborts scan in head exams and tech has to restart since Sept 2.  exams 18962 series 2  18973 series 2  18974 series 2 and 19578  verified that Oct 2 arround 13:30 on  exam 19578 scanner aborted due to max spit.   I could not duplicate tube spits on testing this morning.  ", "LINES ON IMAGES WHEN TUBE IS ANGLED  ", "Customer states scanner hardware stopped  Exam# 30539/1/1    scan/equipment availabe", "Table will not move"]}, "parameters": {"n_topics": 1, "n_words": 10, "col_names": ["description"], "top_n": 1}}', 'result': '{"Score": [100.0], "Topic ID": [1], "Topic": ["SCAN SERIES SCANNER HE

...

{'analyticId': '686b0ab7-f124-4731-9861-b66227d2ee60', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"wotd_data": {"description": ["Table problem", "TABLE DOWN FOOT PEDALS NOT WORKING ", "Previous SR  1-200934622717 Customer States the Table elevation movement without command/Provided PO for billing equipment is available when the FE calls", "scanner hardware error after scan was stopping  rebooted system now ", "CD 10/22 THURS 0700A CST  Having problem with move to scan button always being lit.  Have had this problem continuously over the past month or so.  They say it is something to do with the movement of table.  Exam is started and when trying to move the table it doesn\'t move and the move to scan button just goes off and then back on again.  If it ever gets to the right area then it doesn\'t move to start scan button.  Have to reset the system in order to scan properly again.  This has happened twice this evening where whole machine had to b

{'analyticId': '686b0ab7-f124-4731-9861-b66227d2ee60', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"wotd_data": {"description": ["WHEN THE TUBE ROTATED IT MAKE A CLICKING NOISE. THIS HAPPEN AFTER THE DETECTOR WAS INSTALLED. ", "Research possible system improvements", "Hardware stop scan during fast cals", "Axial encoder is bouncing in slow speed", "Direct connect ; Contact site  PSDB notes to check  cradle or cradle drive assembly for tape  debris  or object jammed  preventing normal cradle drive motion  a miss-adjusted or faulty cradle clutch  a faulty longitudinal encoder  etc. Check gantry rotation for any odd noise.", "Clunking sound from gantry during rotation/and slow-down.  Also  high pitched noise inside gantry. ", "DURING EXAM 31088  EXAM TABLE WOULD NOT MOVE.  DUE TO THIS THE PATIENT HAD TO BE REINJECTED WITH IV CONTRAST.  TABLE WOULD NOT MOVE IN/OUT OF GANTRY DURING SCAN ", "System fails to run tube warm up. Message says: Gantry Found 

{'analyticId': '686b0ab7-f124-4731-9861-b66227d2ee60', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"wotd_data": {"description": ["DVD drive does not work", "CD BURNER INOP", "TRYING TO PRINT A CD LOCALLY USING CD/DVD/USB OPTION BUT IS NOT WORKING ", "Besoin de nettoyer la fenetre mylar.", "**SEE LOGS** We have lost our cursor that we usually see on the scouts. We now can not see what S or I level our mouse is at so that we can adjust our next scout. This happens on all exams since the PM was done on OCt28th 2015.  Also during exam 5123 we landmarked and then went to change it to a different landmark and it would not accept this at either the left or right pad on the side of the gantry. We restarted the scanner and it seems to be accepting our landmarks now on both keypads  but we have not had a patients since the issue.  We have lost our cursor that we usually see on the scouts. We now can not see what S or I level our mouse is at so that we can 

{'analyticId': '686b0ab7-f124-4731-9861-b66227d2ee60', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"wotd_data": {"description": ["Follow Up to Table Issue", "There is a high pitched \\"\\"Whirring\\"\\" noise when advancing the CT table into the ga There is a high pitched \\"\\"Whirring\\"\\" noise when advancing the CT table into the gantry with a patient on board.  Sound is not present when advancing an empty table. ", "INH FE SUPPORT: table allination errors", "customer stated side of the table is coming off", "Customer states something was rammed into the table and now it is not feeding in properly.", "TABLE SQUEAKS WHEN COMING DOWN ", "scanner table not working right ", "Josh Hopper 6157692580 stated the scanner is giving Error for oerssure sensore and  will not allow table to move out nor down.", "**HOUSTON**  Customer states  Unit stops scanning during  scans", "table side panel right rear damaged when lowering table"]}, "parameters": {"n_

{'analyticId': '686b0ab7-f124-4731-9861-b66227d2ee60', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"wotd_data": {"description": [" rt display screen is purple. switched with another CT monitor from another scanner and still purple. ", "I HAVE HAD 2 HARDWARE ERRORS THIS MORNING. I CAN RESUME AND CONTINUE TO SCAN.   PLEASE HELP  THANK YOU ", "LOOKS LIKE ARTIFACT STREAKS THROUGH IMAGES 75-88  NO MOVEMENT  ", "unit stopped in the middle of a scan and got error message stopped scan error trying to send through ilink sorry connectivity has been lost call GE  equipment is available asap", "STREAK ARTIFACT ON BRAIN WITH GANTRY TILT OF 16.5 Exam 13991", "Artifact   2", "Direct connect There is gridded line through about 4 slices", "New note: system paused this morning during tube warmup process with error: 260118213  (60-0316) Spit Max error  PPC board has detected spits number during exposure reached the limit. Original issue: This issue was brought up a

In [97]:
# use topics to get related resolution codes 
counter = 0
for rescode in sr_dict:
    print(rescode)
    print(counter)
    sr_dict[rescode]['related_rescodes'] = pc.run_similarity(sr_dict, rescode)
    counter += 1
    

Reset_UPS
0
{'analyticId': 'b56ac7e1-ce70-45a0-9b68-3c8375039a3c', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"data": {"data_set": {"rescode": ["Reset_UPS", "Troubleshoot_HEMIT_TANK", "No structural problem found", "Configure_ConnectPro", "Reset_the_system_time_clock", "Adjust_gantry_tilt_speed", "Troubleshoot_GANTRY_TOUCH_SENSOR", "Adjust_or_tighten_table_cover_or_touch_switch_or_sensor", "Perform_LFC_and_system_software_reload", "Reset_Image_database", "Re_install_a_software_option", "Check_and_adjust_network_configuration", "Perform_intercom_adjustments", "Troubleshoot_JEDI_INVERTER_AND_HV_TANK", "Troubleshoot_AXIAL_ENCODER", "Clear_recon_image_queue", "Troubleshoot_TABLE_TAPE_SWITCH", "Troubleshoot_GANTRY_CONTROL_PANEL", "Troubleshoot_HOST_COMPUTER", "Troubleshoot_JEDI_HV_TANK", "Cycle_the_Gantry_and_table_power_or_Reset_scan_hardware", "Troubleshoot_SCIM", "Troubleshoot_X_RAY_TUBE", "Cycle_the_Console_power_or_restart_system_software", "Cle

{'analyticId': 'b56ac7e1-ce70-45a0-9b68-3c8375039a3c', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"data": {"data_set": {"rescode": ["Reset_UPS", "Troubleshoot_HEMIT_TANK", "No structural problem found", "Configure_ConnectPro", "Reset_the_system_time_clock", "Adjust_gantry_tilt_speed", "Troubleshoot_GANTRY_TOUCH_SENSOR", "Adjust_or_tighten_table_cover_or_touch_switch_or_sensor", "Perform_LFC_and_system_software_reload", "Reset_Image_database", "Re_install_a_software_option", "Check_and_adjust_network_configuration", "Perform_intercom_adjustments", "Troubleshoot_JEDI_INVERTER_AND_HV_TANK", "Troubleshoot_AXIAL_ENCODER", "Clear_recon_image_queue", "Troubleshoot_TABLE_TAPE_SWITCH", "Troubleshoot_GANTRY_CONTROL_PANEL", "Troubleshoot_HOST_COMPUTER", "Troubleshoot_JEDI_HV_TANK", "Cycle_the_Gantry_and_table_power_or_Reset_scan_hardware", "Troubleshoot_SCIM", "Troubleshoot_X_RAY_TUBE", "Cycle_the_Console_power_or_restart_system_software", "Clean_table_pow

{'analyticId': 'b56ac7e1-ce70-45a0-9b68-3c8375039a3c', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"data": {"data_set": {"rescode": ["Reset_UPS", "Troubleshoot_HEMIT_TANK", "No structural problem found", "Configure_ConnectPro", "Reset_the_system_time_clock", "Adjust_gantry_tilt_speed", "Troubleshoot_GANTRY_TOUCH_SENSOR", "Adjust_or_tighten_table_cover_or_touch_switch_or_sensor", "Perform_LFC_and_system_software_reload", "Reset_Image_database", "Re_install_a_software_option", "Check_and_adjust_network_configuration", "Perform_intercom_adjustments", "Troubleshoot_JEDI_INVERTER_AND_HV_TANK", "Troubleshoot_AXIAL_ENCODER", "Clear_recon_image_queue", "Troubleshoot_TABLE_TAPE_SWITCH", "Troubleshoot_GANTRY_CONTROL_PANEL", "Troubleshoot_HOST_COMPUTER", "Troubleshoot_JEDI_HV_TANK", "Cycle_the_Gantry_and_table_power_or_Reset_scan_hardware", "Troubleshoot_SCIM", "Troubleshoot_X_RAY_TUBE", "Cycle_the_Console_power_or_restart_system_software", "Clean_table_pow

{'analyticId': 'b56ac7e1-ce70-45a0-9b68-3c8375039a3c', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"data": {"data_set": {"rescode": ["Reset_UPS", "Troubleshoot_HEMIT_TANK", "No structural problem found", "Configure_ConnectPro", "Reset_the_system_time_clock", "Adjust_gantry_tilt_speed", "Troubleshoot_GANTRY_TOUCH_SENSOR", "Adjust_or_tighten_table_cover_or_touch_switch_or_sensor", "Perform_LFC_and_system_software_reload", "Reset_Image_database", "Re_install_a_software_option", "Check_and_adjust_network_configuration", "Perform_intercom_adjustments", "Troubleshoot_JEDI_INVERTER_AND_HV_TANK", "Troubleshoot_AXIAL_ENCODER", "Clear_recon_image_queue", "Troubleshoot_TABLE_TAPE_SWITCH", "Troubleshoot_GANTRY_CONTROL_PANEL", "Troubleshoot_HOST_COMPUTER", "Troubleshoot_JEDI_HV_TANK", "Cycle_the_Gantry_and_table_power_or_Reset_scan_hardware", "Troubleshoot_SCIM", "Troubleshoot_X_RAY_TUBE", "Cycle_the_Console_power_or_restart_system_software", "Clean_table_pow

{'analyticId': 'b56ac7e1-ce70-45a0-9b68-3c8375039a3c', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"data": {"data_set": {"rescode": ["Reset_UPS", "Troubleshoot_HEMIT_TANK", "No structural problem found", "Configure_ConnectPro", "Reset_the_system_time_clock", "Adjust_gantry_tilt_speed", "Troubleshoot_GANTRY_TOUCH_SENSOR", "Adjust_or_tighten_table_cover_or_touch_switch_or_sensor", "Perform_LFC_and_system_software_reload", "Reset_Image_database", "Re_install_a_software_option", "Check_and_adjust_network_configuration", "Perform_intercom_adjustments", "Troubleshoot_JEDI_INVERTER_AND_HV_TANK", "Troubleshoot_AXIAL_ENCODER", "Clear_recon_image_queue", "Troubleshoot_TABLE_TAPE_SWITCH", "Troubleshoot_GANTRY_CONTROL_PANEL", "Troubleshoot_HOST_COMPUTER", "Troubleshoot_JEDI_HV_TANK", "Cycle_the_Gantry_and_table_power_or_Reset_scan_hardware", "Troubleshoot_SCIM", "Troubleshoot_X_RAY_TUBE", "Cycle_the_Console_power_or_restart_system_software", "Clean_table_pow

...

{'analyticId': 'b56ac7e1-ce70-45a0-9b68-3c8375039a3c', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"data": {"data_set": {"rescode": ["Reset_UPS", "Troubleshoot_HEMIT_TANK", "No structural problem found", "Configure_ConnectPro", "Reset_the_system_time_clock", "Adjust_gantry_tilt_speed", "Troubleshoot_GANTRY_TOUCH_SENSOR", "Adjust_or_tighten_table_cover_or_touch_switch_or_sensor", "Perform_LFC_and_system_software_reload", "Reset_Image_database", "Re_install_a_software_option", "Check_and_adjust_network_configuration", "Perform_intercom_adjustments", "Troubleshoot_JEDI_INVERTER_AND_HV_TANK", "Troubleshoot_AXIAL_ENCODER", "Clear_recon_image_queue", "Troubleshoot_TABLE_TAPE_SWITCH", "Troubleshoot_GANTRY_CONTROL_PANEL", "Troubleshoot_HOST_COMPUTER", "Troubleshoot_JEDI_HV_TANK", "Cycle_the_Gantry_and_table_power_or_Reset_scan_hardware", "Troubleshoot_SCIM", "Troubleshoot_X_RAY_TUBE", "Cycle_the_Console_power_or_restart_system_software", "Clean_table_pow

{'analyticId': 'b56ac7e1-ce70-45a0-9b68-3c8375039a3c', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"data": {"data_set": {"rescode": ["Reset_UPS", "Troubleshoot_HEMIT_TANK", "No structural problem found", "Configure_ConnectPro", "Reset_the_system_time_clock", "Adjust_gantry_tilt_speed", "Troubleshoot_GANTRY_TOUCH_SENSOR", "Adjust_or_tighten_table_cover_or_touch_switch_or_sensor", "Perform_LFC_and_system_software_reload", "Reset_Image_database", "Re_install_a_software_option", "Check_and_adjust_network_configuration", "Perform_intercom_adjustments", "Troubleshoot_JEDI_INVERTER_AND_HV_TANK", "Troubleshoot_AXIAL_ENCODER", "Clear_recon_image_queue", "Troubleshoot_TABLE_TAPE_SWITCH", "Troubleshoot_GANTRY_CONTROL_PANEL", "Troubleshoot_HOST_COMPUTER", "Troubleshoot_JEDI_HV_TANK", "Cycle_the_Gantry_and_table_power_or_Reset_scan_hardware", "Troubleshoot_SCIM", "Troubleshoot_X_RAY_TUBE", "Cycle_the_Console_power_or_restart_system_software", "Clean_table_pow

{'analyticId': 'b56ac7e1-ce70-45a0-9b68-3c8375039a3c', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"data": {"data_set": {"rescode": ["Reset_UPS", "Troubleshoot_HEMIT_TANK", "No structural problem found", "Configure_ConnectPro", "Reset_the_system_time_clock", "Adjust_gantry_tilt_speed", "Troubleshoot_GANTRY_TOUCH_SENSOR", "Adjust_or_tighten_table_cover_or_touch_switch_or_sensor", "Perform_LFC_and_system_software_reload", "Reset_Image_database", "Re_install_a_software_option", "Check_and_adjust_network_configuration", "Perform_intercom_adjustments", "Troubleshoot_JEDI_INVERTER_AND_HV_TANK", "Troubleshoot_AXIAL_ENCODER", "Clear_recon_image_queue", "Troubleshoot_TABLE_TAPE_SWITCH", "Troubleshoot_GANTRY_CONTROL_PANEL", "Troubleshoot_HOST_COMPUTER", "Troubleshoot_JEDI_HV_TANK", "Cycle_the_Gantry_and_table_power_or_Reset_scan_hardware", "Troubleshoot_SCIM", "Troubleshoot_X_RAY_TUBE", "Cycle_the_Console_power_or_restart_system_software", "Clean_table_pow

{'analyticId': 'b56ac7e1-ce70-45a0-9b68-3c8375039a3c', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"data": {"data_set": {"rescode": ["Reset_UPS", "Troubleshoot_HEMIT_TANK", "No structural problem found", "Configure_ConnectPro", "Reset_the_system_time_clock", "Adjust_gantry_tilt_speed", "Troubleshoot_GANTRY_TOUCH_SENSOR", "Adjust_or_tighten_table_cover_or_touch_switch_or_sensor", "Perform_LFC_and_system_software_reload", "Reset_Image_database", "Re_install_a_software_option", "Check_and_adjust_network_configuration", "Perform_intercom_adjustments", "Troubleshoot_JEDI_INVERTER_AND_HV_TANK", "Troubleshoot_AXIAL_ENCODER", "Clear_recon_image_queue", "Troubleshoot_TABLE_TAPE_SWITCH", "Troubleshoot_GANTRY_CONTROL_PANEL", "Troubleshoot_HOST_COMPUTER", "Troubleshoot_JEDI_HV_TANK", "Cycle_the_Gantry_and_table_power_or_Reset_scan_hardware", "Troubleshoot_SCIM", "Troubleshoot_X_RAY_TUBE", "Cycle_the_Console_power_or_restart_system_software", "Clean_table_pow

{'analyticId': 'b56ac7e1-ce70-45a0-9b68-3c8375039a3c', 'status': 'COMPLETED', 'message': 'Analytic executed successfully.', 'inputData': '{"data": {"data_set": {"rescode": ["Reset_UPS", "Troubleshoot_HEMIT_TANK", "No structural problem found", "Configure_ConnectPro", "Reset_the_system_time_clock", "Adjust_gantry_tilt_speed", "Troubleshoot_GANTRY_TOUCH_SENSOR", "Adjust_or_tighten_table_cover_or_touch_switch_or_sensor", "Perform_LFC_and_system_software_reload", "Reset_Image_database", "Re_install_a_software_option", "Check_and_adjust_network_configuration", "Perform_intercom_adjustments", "Troubleshoot_JEDI_INVERTER_AND_HV_TANK", "Troubleshoot_AXIAL_ENCODER", "Clear_recon_image_queue", "Troubleshoot_TABLE_TAPE_SWITCH", "Troubleshoot_GANTRY_CONTROL_PANEL", "Troubleshoot_HOST_COMPUTER", "Troubleshoot_JEDI_HV_TANK", "Cycle_the_Gantry_and_table_power_or_Reset_scan_hardware", "Troubleshoot_SCIM", "Troubleshoot_X_RAY_TUBE", "Cycle_the_Console_power_or_restart_system_software", "Clean_table_pow

In [98]:
# save into csv file for upload later 
sr_dict

{'Adjust_gantry_alignment_lights': {'para': ['The laser alignment light is flashing on the gantry. Is this of any significance ',
   'SCANNER HARDWARE STOPPED SCAN EXAM 10091  IN THE MIDDLE OF INJECTED A PATIENT FOR A CTA CHEST. PATIENT HAD TO BE RE INJECTED AND RE SCANNED DUE TO THIS ISSUE ',
   'The laser light on the unit needs to be checked.',
   'Service needed to address physicist survey of equipment for the ACR. The lasers alignment light did not appear to be level in the vertical direction across the ACR phantom even though the ACR phantom was level within the gantry. ',
   'NO',
   'QA WAS OFF BY 3 THIS MORNING.  THE LAZER LIGHTS ARE ALSO OFF  QA-NONE-NONE',
   'Trouble with laser lights  gantry display  and landmark.',
   'WANTING TO SET UP LASER FILM PRINTER WITH SCANNER ',
   'Gantry lights flickered during an exam.',
   'Laser centering is now outside QC parameters.  Axial laser is not operating unless table positioned near 0. ',
   'CD Thur 11/3 0900CST Customer states th

In [99]:
import csv
with open('rescodes.csv', mode='w') as csv_file:
    wr = csv.writer(csv_file, delimiter=',')
    for rescode in sr_dict:
        wr.writerow([rescode, sr_dict[rescode]['topics'], sr_dict[rescode]['related_rescodes']])

In [101]:
import pickle
with open('sr_dict.pickle', 'wb') as handle:
    pickle.dump(sr_dict, handle)